# YOLOv7 — Brackish Underwater (Roboflow v2)
Train & evaluate YOLOv7 on the Roboflow dataset, with PyTorch 2.6+ compatibility patches.

## 1) Environment & GPU

In [1]:
import torch, platform
print("Python:", platform.python_version())
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Python: 3.12.11
Torch: 2.8.0+cu126 | CUDA available: True
GPU: Tesla T4


## 2) Install dependencies

In [1]:
# Install dependencies for local Windows environment
import sys, subprocess
def pip_install(pkgs):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip', 'wheel', 'setuptools'])
    subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + pkgs)
try:
    pip_install(['roboflow', 'thop', 'opencv-python-headless', 'matplotlib', 'pyyaml', 'tqdm', 'seaborn', 'pycocotools', 'jedi>=0.16'])
except Exception as e:
    print('Error installing packages:', e)
    print('If pycocotools fails, try installing cython and cocoapi manually:')
    print('pip install cython')
    print('pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI')

## 3) Get YOLOv7 repository

In [ ]:
# One-shot, robust YOLOv7 setup for local Windows/venv
import os, sys, shutil, subprocess, pathlib
REPO_URL = "https://github.com/WongKinYiu/yolov7.git"
REPO_DIR = os.path.join(os.getcwd(), 'yolov7')
# Fresh clone
if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)
subprocess.check_call(["git", "clone", REPO_URL, REPO_DIR])
print("Cloned to:", REPO_DIR)
# Upgrade build tools using the active Python interpreter
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel'])
# Install known-good deps explicitly (skip torch to avoid downgrades/breakage)
BASE_PKGS = [
    'numpy>=1.26', 'pandas>=2.0', 'matplotlib>=3.8', 'seaborn>=0.13',
    'tqdm>=4.66', 'scipy>=1.11', 'Pillow>=10.0', 'requests>=2.31', 'PyYAML>=6.0',
    'opencv-python-headless>=4.8', 'tensorboard>=2.14', 'thop>=0.1.1.post2209072238',
    'protobuf<5',
]
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--no-cache-dir', *BASE_PKGS])
# Try pycocotools wheel first; if not, build from source
def install_pycocotools():
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pycocotools>=2.0.7'])
        print('pycocotools wheel installed.')
        return
    except Exception:
        print('Wheel unavailable; attempting source build...')
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'cython'])
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'])
        print('pycocotools built from official COCOAPI.')
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI'])
        print('pycocotools installed from fallback fork.')
install_pycocotools()
print('
print("\n✅ YOLOv7 environment ready. You can now run train/test scripts under /content/yolov7")


Cloned: /content/yolov7
pycocotools wheel installed.

✅ YOLOv7 environment ready. You can now run train/test scripts under /content/yolov7


## 4) Download dataset from Roboflow (version 2)

In [11]:
# Provided snippet
from roboflow import Roboflow
rf = Roboflow(api_key="tAn8GdtOiAqVLCvRWI7Y")
project = rf.workspace("brad-dwyer").project("brackish-underwater")
version = project.version(2)
dataset = version.download("yolov7")  # returns a Dataset object with .location

DATA_DIR = dataset.location  # e.g., '/content/brackish-underwater-2'
print("Dataset directory:", DATA_DIR)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Brackish-Underwater-2 in yolov7pytorch:: 100%|██████████| 29360/29360 [00:05<00:00, 5521.14it/s] 

Dataset directory: /content/Brackish-Underwater-2


## 5) Locate data.yaml

In [12]:
import os, glob, json
data_yaml = None
# Common locations from Roboflow export
for cand in [
    os.path.join(DATA_DIR, "data.yaml"),
    os.path.join(DATA_DIR, "data.yaml".lower()),
]:
    if os.path.exists(cand):
        data_yaml = cand
        break
if not data_yaml:
    # fallback: search
    cands = glob.glob(os.path.join(DATA_DIR, "**", "data.yaml"), recursive=True)
    if cands: data_yaml = cands[0]

assert data_yaml and os.path.exists(data_yaml), "Could not find data.yaml in the downloaded dataset."
print("Using data.yaml:", data_yaml)

Using data.yaml: /content/Brackish-Underwater-2/data.yaml


### (Optional) Inspect class names from YAML

In [13]:
import yaml
with open(data_yaml) as f:
    y = yaml.safe_load(f)
print("Classes (nc):", y.get("nc"))
print("Names:", y.get("names"))

Classes (nc): 6
Names: ['crab', 'fish', 'jellyfish', 'shrimp', 'small_fish', 'starfish']


## 6) Get base weights (yolov7.pt)

In [ ]:
import os, pathlib, sys, subprocess
# Ensure we're in the repository directory if it exists
repo_dir = os.path.join(os.getcwd(), 'yolov7')
if os.path.exists(repo_dir):
    os.chdir(repo_dir)
weights_path = os.path.join(os.getcwd(), 'yolov7.pt')
if not os.path.exists(weights_path):
    # Download official release weight
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'wget'])
        import wget
        wget.download('https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt', weights_path)
    except Exception:
        # fallback to urllib
        from urllib.request import urlretrieve
        urlretrieve('https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt', weights_path)
print('Weights present:', os.path.abspath(weights_path))
# Return to original cwd
os.chdir(pathlib.Path(repo_dir).parent)


/content/yolov7
Weights present: /content/yolov7/yolov7.pt
/content


## 7) Train
Default settings target a T4 with 640×640 images. Adjust batch size if you see OOM.

In [ ]:
# Train YOLOv7 (URPC-style) BUT:
# - Switch deprecated autocast -> torch.amp.autocast('cuda', enabled=cuda)
# - Cap steps per epoch at 200 (so progress bar shows ~200 iters, not 734)
# - Keep robust data.yaml absolutization & cache cleanup

import os, glob, shutil, yaml, pathlib, re

# -------- 0) PyTorch 2.6+ serialization allowlist --------
import torch, torch.serialization
torch.serialization.add_safe_globals(['numpy._core.multiarray._reconstruct'])

# -------- 1) Limit steps per epoch --------
os.environ["MAX_STEPS_PER_EPOCH"] = "200"  # change if you want a different cap

# -------- 2) cd into repo --------
import sys
repo_dir = os.path.join(os.getcwd(), 'yolov7')
if os.path.exists(repo_dir):
    os.chdir(repo_dir)

# -------- 3) Patch train.py for new autocast + step cap --------
train_py = "/content/yolov7/train.py"
with open(train_py, "r", encoding="utf-8") as f:
    src = f.read()

# (a) Replace deprecated autocast usage
src_new = re.sub(
    r"with\s+amp\.autocast\s*\(\s*enabled\s*=\s*cuda\s*\)\s*:",
    "with torch.amp.autocast('cuda', enabled=cuda):",
    src,
)

# (b) Insert a step cap right after the main dataloader loop begins
# Match the common loop signature in YOLOv7:
loop_pat = r"(for\s+i,\s*\(imgs,\s*targets,\s*paths,\s*_\)\s+in\s+enumerate\(\s*dataloader\s*\)\s*:\s*\n)"
inject = (
    r"\1"
    r"        # --- injected limit to cap steps per epoch ---\n"
    r"        max_steps = int(os.getenv('MAX_STEPS_PER_EPOCH', '0'))\n"
    r"        if max_steps > 0 and i >= max_steps:\n"
    r"            break\n"
)
if re.search(loop_pat, src_new):
    src_new = re.sub(loop_pat, inject, src_new)
else:
    # Fallback: try a slightly looser pattern
    loop_pat2 = r"(for\s+i.*in\s+enumerate\(\s*dataloader\s*\)\s*:\s*\n)"
    src_new = re.sub(loop_pat2, inject, src_new)

if src_new != src:
    with open(train_py, "w", encoding="utf-8") as f:
        f.write(src_new)
    print("Patched train.py: autocast -> torch.amp.autocast + step cap inserted")
else:
    print("No changes applied to train.py (patterns not found or already patched)")

# -------- 4) Settings --------
CFG      = os.getenv('CFG', 'cfg/training/yolov7-tiny.yaml')
WEIGHTS  = os.getenv('WEIGHTS', '')            # '' => train from scratch
EPOCHS   = int(os.getenv('EPOCHS', 50))
IMG_SIZE = int(os.getenv('IMG_SIZE', 640))
BATCH    = int(os.getenv('BATCH', 16))
RUN_NAME = os.getenv('RUN_NAME', 'urpc2019_yolov7')

# Disable W&B unless a key is present
if not os.getenv("WANDB_API_KEY", ""):
    os.environ["WANDB_DISABLED"] = "true"

# -------- 5) Normalize data.yaml to absolute paths --------
if 'dataset' in globals() and hasattr(dataset, 'location'):
    orig_yaml = os.path.join(dataset.location, 'data.yaml')
else:
    orig_yaml = '/content/Brackish-Underwater-2/data.yaml'
assert os.path.exists(orig_yaml), f"data.yaml not found at: {orig_yaml}"

with open(orig_yaml) as f:
    y = yaml.safe_load(f)

base = pathlib.Path(orig_yaml).parent
root = base.parent
ds_name = base.name

def _make_abs_roboflow(p: str):
    if not p: return None
    p = str(p)
    if os.path.isabs(p): return p
    cand = (root / p) if p.startswith(ds_name + "/") else (base / p)
    if not cand.exists():
        a = base / p; b = root / p
        cand = a if a.exists() else (b if b.exists() else cand)
    return str(cand.resolve())

for k in ("train", "val", "valid", "test"):
    if k in y and y[k]:
        y[k] = _make_abs_roboflow(y[k])
if not y.get("val") and y.get("valid"):
    y["val"] = y["valid"]

test_dir = os.path.join(base, "test", "images")
if not y.get("test") and os.path.exists(test_dir):
    y["test"] = test_dir

patched_yaml = "/content/data_abs.yaml"
with open(patched_yaml, "w") as f:
    yaml.safe_dump(y, f, sort_keys=False)

print("Using data.yaml:", patched_yaml)
print("train:", y.get("train"))
print("val:  ", y.get("val"))
print("test: ", y.get("test"))

assert y.get("train") and os.path.exists(y["train"]), f"Train path missing: {y.get('train')}"
assert y.get("val") and os.path.exists(y["val"]), f"Val path missing: {y.get('val')}"

# -------- 6) Clear stale caches --------
for d in [str(base), "/content"]:
    for p in glob.glob(os.path.join(d, "**", "*.cache"), recursive=True):
        try:
            os.remove(p)
            print("Removed cache:", p)
        except Exception:
            pass

# -------- 7) Launch training (note: --img is correct flag for yolov7) --------
print('\nLaunching training with step cap =', os.environ['MAX_STEPS_PER_EPOCH'], '...\n')
import shlex, subprocess
train_cmd = [sys.executable, 'train.py', '--workers', '8', '--device', '0', '--batch-size', str(BATCH), '--epochs', str(EPOCHS), '--img', str(IMG_SIZE), str(IMG_SIZE), '--data', patched_yaml, '--cfg', CFG, '--weights', WEIGHTS or 'yolov7.pt', '--name', RUN_NAME]
print('Running:', ' '.join(shlex.quote(a) for a in train_cmd))
subprocess.check_call(train_cmd)

/content/yolov7
Patched train.py: autocast -> torch.amp.autocast + step cap inserted
Using data.yaml: /content/data_abs.yaml
train: /content/Brackish-Underwater-2/train/images
val:   /content/Brackish-Underwater-2/valid/images
test:  /content/Brackish-Underwater-2/test/images
Removed cache: /content/Brackish-Underwater-2/valid/labels.cache
Removed cache: /content/Brackish-Underwater-2/train/labels.cache

Launching training with step cap = 200 ...

2025-08-31 19:36:23.509494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756668983.529321    9340 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756668983.535290    9340 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already b

### 8) Post-Training Evaluation
This step runs validation using the best weights from training.  
It will report:

- Number of layers  
- Total number of parameters  
- GFLOPS  
- FPS  
- Mean Average Precision (mAP)  
- Average Precision (per class)  
- Recall  
- F1 score  
- Confusion matrix (saved as image)  
- Architecture details summary

In [ ]:
# === Post-training evaluation & metrics harvest (YOLOv7) ===
import os, sys, glob, shlex, subprocess, re, csv, json, pathlib
from collections import OrderedDict

y7 = "/content/yolov7"
RUN_NAME = "urpc2019_yolov7"   # change if you used a different --name
DATA_YAML = "/content/data_abs.yaml"

# 1) Locate best.pt (prefer run name, fallback to exp*)
train_dirs = sorted(
    glob.glob(os.path.join(y7, "runs", "train", f"{RUN_NAME}*")) +
    glob.glob(os.path.join(y7, "runs", "train", "exp*")),
    key=os.path.getmtime
)
assert train_dirs, "No training runs found."
last_train = train_dirs[-1]
best = os.path.join(last_train, "weights", "best.pt")
if not os.path.exists(best):
    best = os.path.join(last_train, "weights", "last.pt")
assert os.path.exists(best), f"No weights found at {best}"
print("Using weights:", best)
print("Train dir:", last_train)

# 2) Reprint model summary (#layers, #params, GFLOPS) by building the model once
#    (works even if test didn't print it)
sys.path.insert(0, y7)
from models.experimental import attempt_load
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = attempt_load(best, map_location=device)  # our repo already handles weights_only in your earlier patches
model.eval()
# Count layers & params
num_layers = sum(1 for _ in model.modules())
num_params = sum(p.numel() for p in model.parameters())
# Try to compute MACs/Flops via forward hooks is heavy; rely on YOLOv7 printed GFLOPS if available in log.
# We'll parse training log for GFLOPS line as fallback.
gflops = None
for log_path in [os.path.join(last_train, "opt.yaml"),
                 os.path.join(last_train, "results.txt"),
                 os.path.join(y7, "runs", "train", RUN_NAME, "results.txt")]:
    pass  # placeholders

# Parse GFLOPS from the console logs captured in results.txt (YOLOv7 writes one line with 'GFLOPS')
gflops_re = re.compile(r"Model Summary:\s*\d+\s*layers,\s*[\d,]+\s*parameters.*?([0-9]*\.?[0-9]+)\s*GFLOPS", re.I)
gflops_scan_files = []
# Try typical train console capture (not always saved). We’ll scan recent W&B logs and the notebook output dir.
cand_logs = glob.glob(os.path.join(last_train, "*.log")) + glob.glob(os.path.join(y7, "wandb", "run-*", "logs", "debug.log"))
for p in cand_logs[::-1]:
    try:
        with open(p, "r", errors="ignore") as f:
            m = gflops_re.search(f.read())
        if m:
            gflops = float(m.group(1))
            break
    except Exception:
        pass

# 3) Run validation; capture output to parse speed (FPS) & ensure plots are saved (confusion matrix)
val_cmd = [
    sys.executable, os.path.join(y7, "test.py"),
    "--weights", best,
    "--data", DATA_YAML,
    "--img-size", "640",
    "--batch-size", "16",
    "--conf-thres", "0.001",
    "--iou-thres", "0.65",
    "--task", "val",
    "--save-json",
    "--plots",
    "--device", "0" if torch.cuda.is_available() else "cpu"
]
print("Running val:", " ".join(shlex.quote(a) for a in val_cmd))
proc = subprocess.run(val_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)

# 4) Where test results live
test_dirs = sorted(glob.glob(os.path.join(y7, "runs", "test", "exp*")), key=os.path.getmtime)
assert test_dirs, "No test runs found; check the val logs above."
last_test = test_dirs[-1]
print("Test dir:", last_test)

# 5) Parse Speed for FPS (line like: 'Speed: 1.4ms pre-process, 3.1ms inference, 1.0ms NMS per image')
speed_line = None
for line in proc.stdout.splitlines()[::-1]:
    if "Speed:" in line and "inference" in line and "NMS" in line:
        speed_line = line.strip()
        break

fps = None
if speed_line:
    # Extract inference and NMS ms
    m = re.search(r"inference,\s*([0-9.]+)ms.*?NMS\s*([0-9.]+)ms", speed_line)
    if m:
        inf_ms = float(m.group(1))
        nms_ms = float(m.group(2))
        per_img_ms = inf_ms + nms_ms
        if per_img_ms > 0:
            fps = 1000.0 / per_img_ms

# 6) Parse metrics from results.txt (YOLOv7 writes a CSV-like txt with header)
results_txt = os.path.join(last_test, "results.txt")
metrics = OrderedDict()
if os.path.exists(results_txt):
    with open(results_txt, "r") as f:
        rows = [r.strip() for r in f.readlines() if r.strip()]
    # The last row has the final metrics
    header = rows[0].split()
    vals   = rows[-1].split()
    # YOLOv7 results.txt header example:  epoch, GIoU, obj, cls, total, P, R, mAP@.5, mAP@.5:.95
    # We’ll map known names if present:
    try:
        idx = {h:i for i,h in enumerate(header)}
        def getf(name, default=None):
            return float(vals[idx[name]]) if name in idx else default
        metrics["Precision"] = getf("P")
        metrics["Recall"] = getf("R")
        metrics["mAP@0.5"] = getf("mAP@.5") or getf("mAP@0.5")
        metrics["mAP@0.5:0.95"] = getf("mAP@.5:.95") or getf("mAP@0.5:0.95")
    except Exception:
        pass

# 7) Per-class AP, Confusion Matrix, F1, etc.
# YOLOv7 writes per-class PR-curves and confusion matrix to the test dir.
conf_mat_png = glob.glob(os.path.join(last_test, "*confusion_matrix.png"))
conf_mat_png = conf_mat_png[0] if conf_mat_png else None
# Per-class stats CSV (if present)
per_class_csv = glob.glob(os.path.join(last_test, "labels_*.csv"))  # sometimes saved; else skip
per_class_ap = {}
if per_class_csv:
    try:
        with open(per_class_csv[0], "r") as f:
            rdr = csv.DictReader(f)
            for row in rdr:
                cls = row.get("class") or row.get("Class") or row.get("name") or row.get("Name")
                ap50 = row.get("AP@0.50") or row.get("AP@.50") or row.get("AP50")
                if cls and ap50:
                    per_class_ap[cls] = float(ap50)
    except Exception:
        pass

# F1: If not directly provided, approximate best-F1 from PR curve isn’t trivial to re-derive here.
# We’ll compute F1 from global P/R if available (this is just one operating point).
f1 = None
if metrics.get("Precision") is not None and metrics.get("Recall") is not None:
    P = metrics["Precision"]; R = metrics["Recall"]
    if (P + R) > 0:
        f1 = 2 * P * R / (P + R)

# 8) Print summary
print("\n=== Summary ===")
print(f"Architecture: {model.yaml.get('arch','YOLOv7')} ({model.yaml.get('depth_multiple','?')} depth, {model.yaml.get('width_multiple','?')} width)")
print(f"Layers: {num_layers}")
print(f"Parameters: {num_params:,}")
print(f"GFLOPS: {gflops if gflops is not None else 'see training log'}")
print(f"FPS (approx from val speed): {fps:.2f}" if fps else "FPS: not available (no speed line parsed)")
if metrics:
    for k,v in metrics.items():
        if v is not None:
            print(f"{k}: {v:.4f}")
        else:
            print(f"{k}: N/A")
if f1 is not None:
    print(f"F1 (from global P/R): {f1:.4f}")
else:
    print("F1: N/A")

if per_class_ap:
    print("\nAverage Precision per class (AP@0.50):")
    for k,v in per_class_ap.items():
        print(f"  {k}: {v:.4f}")
else:
    print("\nPer-class AP file not found; check plots in:", last_test)

print("\nConfusion matrix image:", conf_mat_png if conf_mat_png else "not found (check test dir)")
print("Test directory:", last_test)


In [ ]:
import os
import glob
import random
import shutil

# Define the path to your Brackish test images (update to your local path, e.g., 'C:/path/to/Brackish-Underwater-2/test/images')
test_images_path = os.path.join(os.getcwd(), 'Brackish-Underwater-2', 'test', 'images')

# Get a list of all image files in the test directory
all_test_images = glob.glob(os.path.join(test_images_path, '*.jpg')) + \
                  glob.glob(os.path.join(test_images_path, '*.jpeg')) + \
                  glob.glob(os.path.join(test_images_path, '*.png'))

# Check if there are enough test images
if len(all_test_images) < 50:
    print(f"⚠️ Warning: Only {len(all_test_images)} test images found. Zipping all of them.")
    images_to_zip = all_test_images
else:
    # Select 50 random test images
    images_to_zip = random.sample(all_test_images, 50)

# Create a temporary directory to store the selected images
zip_dir = '/content/selected_brackish_test_images'
os.makedirs(zip_dir, exist_ok=True)

# Copy the selected images to the temporary directory
for img_path in images_to_zip:
    shutil.copy(img_path, zip_dir)

# Zip the temporary directory
zip_filename = 'brackish_selected_test_images.zip'
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', zip_dir)

print(f'✅ Created {zip_filename} in the current working directory. Transfer it from the notebook's folder to your local machine if needed.')


In [ ]:
# You can use this cell to download the zip file after the previous cell finishes
from google.colab import files
files.download('brackish_selected_test_images.zip')